In [ ]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401
from IPython.display import HTML
from utils import time_vector

plt.style.use('seaborn-v0_8')


# Control cinemático de un manipulador espacial de 4 GDL (x, y, z, pitch)


## 1. Parámetros


In [ ]:
params_4dof = {
    'L1': 0.4,  # desplazamiento vertical de la base
    'L2': 0.5,
    'L3': 0.35
}

sim_config_4dof = {
    'kp': np.diag([8.0, 8.0, 8.0, 5.0]),
    't0': 0.0,
    'tf': 10.0,
    'n_steps': 900
}

t_4dof, dt_4dof = time_vector(sim_config_4dof)


## 2. Cinemática directa


In [ ]:
def fkine_4dof(q, params):
    q1, q2, q3, q4 = q
    L1, L2, L3 = params['L1'], params['L2'], params['L3']

    c1, s1 = np.cos(q1), np.sin(q1)
    c2, s2 = np.cos(q2), np.sin(q2)
    c23, s23 = np.cos(q2 + q3), np.sin(q2 + q3)

    reach = L2 * c2 + L3 * c23
    x = c1 * reach
    y = s1 * reach
    z = L1 + L2 * s2 + L3 * s23
    pitch = q2 + q3 + q4
    return np.array([x, y, z, pitch])

fkine_4dof(np.array([0.0, 0.2, -0.3, 0.1]), params_4dof)


## 3. Jacobiano


In [ ]:
def jacobian_4dof(q, params):
    q1, q2, q3, q4 = q
    L1, L2, L3 = params['L1'], params['L2'], params['L3']

    c1, s1 = np.cos(q1), np.sin(q1)
    c2, s2 = np.cos(q2), np.sin(q2)
    c23, s23 = np.cos(q2 + q3), np.sin(q2 + q3)
    reach = L2 * c2 + L3 * c23
    reach_s = -L2 * s2 - L3 * s23

    J = np.array([
        [-s1 * reach, c1 * reach_s, c1 * (-L3 * s23), 0.0],
        [ c1 * reach, s1 * reach_s, s1 * (-L3 * s23), 0.0],
        [ 0.0,        L2 * c2 + L3 * c23, L3 * c23, 0.0],
        [ 0.0, 1.0, 1.0, 1.0]
    ])
    return J

jacobian_4dof(np.array([0.0, 0.2, -0.3, 0.1]), params_4dof)


### Derivación simbólica


In [ ]:
q1_s, q2_s, q3_s, q4_s, L1_s, L2_s, L3_s = sp.symbols('q1 q2 q3 q4 L1 L2 L3')
reach = L2_s * sp.cos(q2_s) + L3_s * sp.cos(q2_s + q3_s)
px = sp.cos(q1_s) * reach
py = sp.sin(q1_s) * reach
pz = L1_s + L2_s * sp.sin(q2_s) + L3_s * sp.sin(q2_s + q3_s)
pitch = q2_s + q3_s + q4_s
J_sym = sp.Matrix([px, py, pz, pitch]).jacobian([q1_s, q2_s, q3_s, q4_s])
sp.simplify(J_sym)


## 4. Trayectoria objetivo en el plano XZ


In [ ]:
radius_4dof = 0.18
center_xz = (0.55, 0.55)  # (x, z)
y_ref = 0.15
omega_4dof = 0.8

t = t_4dof
cx, cz = center_xz
xd = cx + radius_4dof * np.cos(omega_4dof * t)
yd = np.full_like(xd, y_ref)
zd = cz + radius_4dof * np.sin(omega_4dof * t)
xdp = -radius_4dof * omega_4dof * np.sin(omega_4dof * t)
ydp = np.zeros_like(xd)
zdp = radius_4dof * omega_4dof * np.cos(omega_4dof * t)
pitch_d = np.unwrap(np.arctan2(zdp, xdp))
pitch_dp = np.gradient(pitch_d, t)

Xd_4dof = np.vstack((xd, yd, zd, pitch_d))
Vd_4dof = np.vstack((xdp, ydp, zdp, pitch_dp))
Xd_4dof[:, :3]


## 5. Control cinemático


In [ ]:
def kinematic_control_step_4dof(q, params, xd, xdp, kp):
    x = fkine_4dof(q, params)
    J = jacobian_4dof(q, params)
    dq = np.linalg.solve(J, xdp + kp @ (xd - x))
    return dq, x

kinematic_control_step_4dof(np.array([0.0, 0.2, -0.3, 0.1]), params_4dof, Xd_4dof[:, 0], Vd_4dof[:, 0], sim_config_4dof['kp'])


## 6. Simulación


In [ ]:
def simulate_4dof_control(q0, params, Xd, Vd, t, kp):
    n = t.size
    dt = t[1] - t[0]
    q = np.zeros((4, n))
    X = np.zeros((4, n))
    q[:, 0] = q0
    X[:, 0] = fkine_4dof(q0, params)
    for k in range(1, n):
        dq, _ = kinematic_control_step_4dof(q[:, k-1], params, Xd[:, k-1], Vd[:, k-1], kp)
        q[:, k] = q[:, k-1] + dt * dq
        X[:, k] = fkine_4dof(q[:, k])
    return q, X

q0_4dof = np.array([0.0, 0.3, -0.4, 0.2])
theta_4dof, X_4dof = simulate_4dof_control(q0_4dof, params_4dof, Xd_4dof, Vd_4dof, t_4dof, sim_config_4dof['kp'])
X_4dof[:, :3]


In [ ]:
pos_error_4dof = np.linalg.norm(Xd_4dof[:3] - X_4dof[:3], axis=0)
pitch_error_4dof = np.angle(np.exp(1j * (Xd_4dof[3] - X_4dof[3])))
max_pos_error_4dof = float(np.max(pos_error_4dof))
max_pitch_error_4dof = float(np.max(np.abs(pitch_error_4dof)))
print(f'Error máximo de posición: {max_pos_error_4dof:.4f} m')
print(f'Error máximo de pitch: {max_pitch_error_4dof:.4f} rad')
assert max_pos_error_4dof < 6e-2
assert max_pitch_error_4dof < 8e-2


## 7. Visualización


In [ ]:
fig = plt.figure(figsize=(7, 5))
ax = fig.add_subplot(111, projection='3d')
ax.plot(Xd_4dof[0], Xd_4dof[1], Xd_4dof[2], '--', label='Trayectoria deseada')
ax.plot(X_4dof[0], X_4dof[1], X_4dof[2], label='Seguimiento')
ax.set_xlabel('x [m]')
ax.set_ylabel('y [m]')
ax.set_zlabel('z [m]')
ax.set_title('Seguimiento de trayectoria 3D')
ax.legend(loc='upper right')
plt.tight_layout()


In [ ]:
plt.figure(figsize=(6, 3))
plt.plot(t_4dof, pos_error_4dof, label='Error posicional')
plt.plot(t_4dof, np.rad2deg(pitch_error_4dof), label='Error de pitch [deg]')
plt.xlabel('Tiempo [s]')
plt.ylabel('Error')
plt.title('Errores de seguimiento')
plt.legend()
plt.grid(True)
plt.tight_layout()


### Animación


In [ ]:
def animate_4dof(theta, X, Xd, params, skip=3):
    L1, L2, L3 = params['L1'], params['L2'], params['L3']

    fig = plt.figure(figsize=(7, 6))
    ax = fig.add_subplot(111, projection='3d')
    ax.set_title('Robot espacial de 4 GDL')

    margin = 0.15
    x_all = np.concatenate([X[0], Xd[0]])
    y_all = np.concatenate([X[1], Xd[1]])
    z_all = np.concatenate([X[2], Xd[2], np.array([0.0, L1 + L2 + L3])])
    ax.set_xlim(np.min(x_all) - margin, np.max(x_all) + margin)
    ax.set_ylim(np.min(y_all) - margin, np.max(y_all) + margin)
    ax.set_zlim(np.min(z_all) - margin, np.max(z_all) + margin)
    ax.set_xlabel('x [m]')
    ax.set_ylabel('y [m]')
    ax.set_zlabel('z [m]')

    desired_line, = ax.plot(Xd[0], Xd[1], Xd[2], '--', linewidth=1.2, label='Trayectoria deseada')
    trail_line,   = ax.plot([], [], [], ':', linewidth=1.5, label='Rastro EE')
    link_line,    = ax.plot([], [], [], 'k-o', linewidth=3)
    orient_quiver = None
    ax.legend(loc='upper right')

    def joint_positions(q):
        q1, q2, q3, q4 = q
        c1, s1 = np.cos(q1), np.sin(q1)
        c2, s2 = np.cos(q2), np.sin(q2)
        c23, s23 = np.cos(q2 + q3), np.sin(q2 + q3)
        base = np.array([0.0, 0.0, 0.0])
        shoulder = np.array([0.0, 0.0, L1])
        reach2 = L2 * np.array([c1 * c2, s1 * c2, s2])
        elbow = shoulder + reach2
        reach3 = L3 * np.array([c1 * c23, s1 * c23, s23])
        wrist = elbow + reach3
        pitch = q2 + q3 + q4
        tool_dir = np.array([c1 * np.cos(pitch), s1 * np.cos(pitch), np.sin(pitch)])
        return base, shoulder, elbow, wrist, tool_dir

    trail_points = []

    def update(frame):
        nonlocal orient_quiver
        k = frame * skip
        if k >= theta.shape[1]:
            k = theta.shape[1] - 1
        base, shoulder, elbow, wrist, tool_dir = joint_positions(theta[:, k])
        xs = [base[0], shoulder[0], elbow[0], wrist[0]]
        ys = [base[1], shoulder[1], elbow[1], wrist[1]]
        zs = [base[2], shoulder[2], elbow[2], wrist[2]]
        link_line.set_data(xs, ys)
        link_line.set_3d_properties(zs)

        trail_points.append(wrist)
        trail_arr = np.array(trail_points)
        trail_line.set_data(trail_arr[:, 0], trail_arr[:, 1])
        trail_line.set_3d_properties(trail_arr[:, 2])

        if orient_quiver is not None:
            orient_quiver.remove()
        orient_quiver = ax.quiver(
            wrist[0], wrist[1], wrist[2],
            tool_dir[0], tool_dir[1], tool_dir[2],
            length=0.15, color='tab:red'
        )
        return link_line, trail_line, orient_quiver

    frames = max(1, theta.shape[1] // skip)
    anim = FuncAnimation(fig, update, frames=frames, interval=40, blit=False, repeat=False)
    plt.close(fig)
    return anim

anim_4dof = animate_4dof(theta_4dof, X_4dof, Xd_4dof, params_4dof)
HTML(anim_4dof.to_jshtml())
